<a href="https://colab.research.google.com/github/YejinK/contrastive-learning-yk-pg/blob/Paul/Colab_SimCSE_unsup_bert_uncased_train_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Unsupervised_bert_uncased training on 5M tweets dataset

This training sequence utilizes base SimCSE hyperparameters using the wiki1m dataset used in the paper

--- 

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [ ]:
!pip install faiss-gpu
!pip install torch==1.7.1+cu110 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85.5 MB 1.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1 MB 2.0 MB/s eta 0:02:41tcmalloc: large alloc 1147494400 bytes == 0x3a560000 @  0x7f43c1861615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |█████████████████████████████▏  | 1055.7 MB 1.1 MB/s eta 0:01:29tcmalloc: large alloc 1434370048 bytes == 0x7ebb6000 @  0x7f43c1861615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8

In [ ]:
!git clone https://github.com/princeton-nlp/SimCSE.git
%cd SimCSE
!pip install -r requirements.txt
!python setup.py install

Cloning into 'SimCSE'...
remote: Enumerating objects: 434, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 434 (delta 43), reused 36 (delta 34), pack-reused 378
Receiving objects: 100% (434/434), 41.47 MiB | 49.78 MiB/s, done.
Resolving deltas: 100% (210/210), done.
/content/SimCSE
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 4.6 MB/s 
     |████████████████████████████████| 451 kB 66.0 MB/s 
     |████████████████████████████████| 11.6 MB 62.8 MB/s 
     |████████████████████████████████| 880 kB 64.1 MB/s 
     |████████████████████████████████| 2.9 MB 64.9 MB/s 
     |████████████████████████████████| 115 kB 55.1 MB/s 
     |████████████████████████████████| 212 kB 91.0 MB/s 
     |████████████████████████████████| 182 kB 91.5 MB/s 
     |████████████████████████████████| 127 kB 86.9 MB/s 
     |███████████████████████

In [ ]:
!bash /content/SimCSE/data/download_wiki.sh 
%cd /content/SimCSE/SentEval/data/downstream/ 
!bash /content/SimCSE/SentEval/data/downstream/download_dataset.sh 
%cd /content/SimCSE

--2022-11-26 22:13:44--  https://huggingface.co/datasets/princeton-nlp/datasets-for-simcse/resolve/main/wiki1m_for_simcse.txt
Resolving huggingface.co (huggingface.co)... 34.227.196.80, 54.147.99.175, 2600:1f18:147f:e800:3df1:c2fc:20aa:9b45, ...
Connecting to huggingface.co (huggingface.co)|34.227.196.80|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/datasets/princeton-nlp/datasets-for-simcse/7b1825863a99aa76479b0456f7c210539dfaeeb69598b41fb4de4f524dd5a706?response-content-disposition=attachment%3B%20filename%3D%22wiki1m_for_simcse.txt%22&Expires=1669760024&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL2RhdGFzZXRzL3ByaW5jZXRvbi1ubHAvZGF0YXNldHMtZm9yLXNpbWNzZS83YjE4MjU4NjNhOTlhYTc2NDc5YjA0NTZmN2MyMTA1MzlkZmFlZWI2OTU5OGI0MWZiNGRlNGY1MjRkZDVhNzA2P3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249YXR0YWNobWVudCUzQiUyMGZpbGVuYW1lJTNEJTIyd2lraTFtX2Zvcl9zaW1jc2UudHh0JTIyIiwiQ29uZGl0aW9uIjp7IkRhdGVMZXNzVG

In [ ]:
from psutil import virtual_memory
from time import time
env_state = {}
env_state['env_ram_gb'] = virtual_memory().total / 1e9

In [ ]:
tweet_path = '/content/drive/MyDrive/CSCI6907-2022-NeuralNetworks/tweets.txt'

In [ ]:
env_state['file_path'] = tweet_path
start_time = time()

In [ ]:
!python train.py \
    --model_name_or_path bert-base-uncased \
    --train_file /content/drive/MyDrive/CSCI6907-2022-NeuralNetworks/tweets.txt \
    --output_dir result/unsup-bert-base-tweets\
    --num_train_epochs 1 \
    --per_device_train_batch_size 64 \
    --learning_rate 3e-5 \
    --max_seq_length 32 \
    --evaluation_strategy steps \
    --metric_for_best_model stsb_spearman \
    --load_best_model_at_end \
    --eval_steps 125 \
    --pooler_type cls \
    --mlp_only_train \
    --overwrite_output_dir \
    --temp 0.05 \
    --do_train \
    --do_eval \
    --fp16 \
    "$@"

env_state['runtime_tweet_train_secs'] = start_time - time()

2022-11-26 22:13:50.922546: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Extracting data files: 100% 1/1 [00:00<00:00, 19.02it/s]
Dataset text downloaded and prepared to /content/SimCSE/./data/text/default-37b4eeb8c7f0d771/0.0.0/99cc88223027054f94ce0c7fd69d10eb172910fa0615671283a3c8e5e7af2f9c. Subsequent calls will reuse this data.
100% 1/1 [00:00<00:00, 22.25it/s]
[INFO|file_utils.py:1272] 2022-11-26 22:14:05,938 >> https://huggingface.co/bert-base-uncased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmprh5g5com
Downloading: 100% 570/570 [00:00<00:00, 733kB/s]
[INFO|file_utils.py:1276] 2022-11-26 22:14:06,062 >> storing https://huggingface.co/bert-base-uncased/resolve/main/config.json i

In [ ]:
%cp -r '/content/SimCSE/result' '/content/drive/MyDrive/CSCI6907-2022-NeuralNetworks'

In [ ]:
!python evaluation.py \
    --model_name_or_path /content/SimCSE/result/unsup-bert-base-tweets \
    --pooler cls \
    --task_set sts \
    --mode test

Some weights of BertModel were not initialized from the model checkpoint at /content/SimCSE/result/unsup-bert-base-tweets and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2022-11-27 02:09:52,970 : ***** Transfer task : STS12 *****


./SentEval/senteval/sts.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
./SentEval/senteval/sts.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=ob